# Ames Housing Data Preprocessing and Feature Selection

This project aims to predict housing prices in Ames, Iowa based on the famous Ames Iowa Housing Dataset from the Ames City Assessor's Office, made available by Dean De Cock of Truman University in 2011. Professor De Cock's original paper can be found [here.](http://jse.amstat.org/v19n3/decock.pdf)

Presentation Slides for this project can be found [here.](https://www.beautiful.ai/player/-N4J5UYshyuRtwl5G4I7)

This notebook includes python code for data extraction, cleaning and transformation. Model Fitting and Analysis code can be found in the Ames_Models notebook.

# Sections and steps

- <a href="#IMP">Reading in Data</a><br>
- <a href="#SP">Second Pass - Bivariate Analysis and Statistics</a><br>
    - Pearson's correlation for numerical variables
    - ANOVA and condensing for categorical variables
- <a href="#PRE">Preparing Data for Models</a><br>
    - Linear Models
        -dummify categorical variables
    - Non Linear Models
        -label encode categorical variables
- <a href="#PRE">First Pass Linear Models</a><br>
- <a href="#PRE">Feature Selection</a><br>
- <a href="#EXP">Exporting Cleaned Datasets</a><br>

<p><a name="IMP"></a></p>

## Importing Packages, Reading in Data

In [1]:
#packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.formula.api import ols

#helper module
from helper_module import *

#setting options to view dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#warnings
import warnings
warnings.filterwarnings("ignore")

#helper module functionality
%load_ext autoreload
%autoreload 2

In [2]:
#testing to make sure helper module loaded
test_helpermod()

'Yes! Module loaded successfully!'

#### Reading in train data

In [3]:
train = pd.read_csv('./data/cleaned/train_c.csv')
train.head()
train.drop(['Unnamed: 0'], axis=1, inplace=True)
print(f"Read in Ames Housing Train Data - Shape: {train.shape}")

Read in Ames Housing Train Data - Shape: (1451, 68)


In [4]:
train.head()

,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,BldgType,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MoSold,SaleType,SaleCondition,SalePrice,log_SalePrice,AgeHome,YrsSnRmdl,TotBthrm,TotFloors,NeighbQual,HasFireplace,HasAlley,HasFence,HasPool,HasGarage,NearNeg,NearPos
0,RL,65,8450,Reg,Lvl,Inside,Gtl,1Fam,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,854,0,1710,3,1,Gd,Typ,NoFireplace,Attchd,RFn,2,TA,TA,Y,0,61,0,0,0,2,WD,Normal,208500,12.247694,5,5,3.5,1,TA,1,0,0,0,1,0,0
1,RL,80,9600,Reg,Lvl,FR2,Gtl,1Fam,6,8,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,0,0,1262,3,1,TA,Typ,TA,Attchd,RFn,2,TA,TA,Y,298,0,0,0,0,5,WD,Normal,181500,12.109011,31,31,2.5,1,Ex,1,0,0,0,1,1,0
2,RL,68,11250,IR1,Lvl,Inside,Gtl,1Fam,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,162,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,866,0,1786,3,1,Gd,Typ,TA,Attchd,RFn,2,TA,TA,Y,0,42,0,0,0,9,WD,Normal,223500,12.317167,7,6,3.5,1,TA,1,0,0,0,1,0,0
3,RL,60,9550,IR1,Lvl,Corner,Gtl,1Fam,7,5,Gable,CompShg,Wd Sdng,Wd Shng,None,0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,756,0,1717,3,1,Gd,Typ,Gd,Detchd,Unf,3,TA,TA,Y,0,35,272,0,0,2,WD,Abnorml,140000,11.849398,91,36,2.0,1,Ex,1,0,0,0,1,0,0
4,RL,84,14260,IR1,Lvl,FR2,Gtl,1Fam,8,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1053,0,2198,4,1,Gd,Typ,TA,Attchd,RFn,3,TA,TA,Y,192,84,0,0,0,12,WD,Normal,250000,12.429216,8,8,3.5,1,Ex,1,0,0,0,1,0,0


In [5]:
#looking at the datatypes of each variable
#train.dtypes

In [6]:
#making sure the binary variables are categorical
for var in ['HasFireplace', 'HasFence', 'HasGarage', 'NearNeg']:
    train[var]= train[var].astype(str)

<p><a name="SP"></a></p>

## Second Pass - Bivariate Analysis

### Pearson's correlation for numerical variables

Now that we have addressed all missing data, and removed outliers, we can re-run some bivariate statistical tests to help us glean which features to select.

In [8]:
#selecting numerical variables
num_var = train.select_dtypes('number').columns.tolist()
#num_var.remove('index')
#num_var.remove('Id')
num_var.remove('SalePrice')
num_var.remove('log_SalePrice')

In [9]:
len(num_var)

28

In [10]:
#pearson's correlation
lose = []
def my_pear(numvar_list):
    keep = []
    for item in numvar_list:
        corr = pearsonr(train[item], train['log_SalePrice'])
        print('Pearsons correlation for ' + item + ' (r):', round(corr[0],3))
        if abs(round(corr[0],3)) > .15:
            keep.append(item)
    return keep

In [11]:
num_var_trim = my_pear(num_var)

Pearsons correlation for LotFrontage (r): 0.33
Pearsons correlation for LotArea (r): 0.36
Pearsons correlation for OverallQual (r): 0.821
Pearsons correlation for OverallCond (r): -0.037
Pearsons correlation for MasVnrArea (r): 0.426
Pearsons correlation for BsmtFinSF1 (r): 0.379
Pearsons correlation for BsmtFinSF2 (r): -0.0
Pearsons correlation for BsmtUnfSF (r): 0.226
Pearsons correlation for TotalBsmtSF (r): 0.64
Pearsons correlation for 2ndFlrSF (r): 0.309
Pearsons correlation for LowQualFinSF (r): -0.037
Pearsons correlation for GrLivArea (r): 0.718
Pearsons correlation for BedroomAbvGr (r): 0.204
Pearsons correlation for KitchenAbvGr (r): -0.148
Pearsons correlation for GarageCars (r): 0.681
Pearsons correlation for WoodDeckSF (r): 0.332
Pearsons correlation for OpenPorchSF (r): 0.329
Pearsons correlation for EnclosedPorch (r): -0.148
Pearsons correlation for 3SsnPorch (r): 0.057
Pearsons correlation for ScreenPorch (r): 0.125
Pearsons correlation for MoSold (r): 0.063
Pearsons c

In [13]:
print('Starting with ' + str(len(num_var)) + ' numerical variables.')
print('Reducing to ' + str(len(num_var_trim)) + ' numerical variables.')

Starting with 28 numerical variables.
Reducing to 16 numerical variables.


### ANOVA and condensing for categorical variables

In [14]:
#selecting categorical variables
cat_var = train.select_dtypes('O').columns.tolist()

In [15]:
len(cat_var)

38

In [25]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

def my_anova(catvar_list):
    C_list = []
    keep = []
    for item in catvar_list:
        temp_ = 'log_SalePrice ~ C(' + item + ')'
        model = ols(temp_, data = train).fit()
        anova_tbl = sm.stats.anova_lm(model, typ=2)
        effect_size = anova_tbl['sum_sq'][0]/(anova_tbl['sum_sq'][0]+anova_tbl['sum_sq'][1])
        
        p_val = round(anova_tbl['PR(>F)'][0], 5)
        print('ANOVA p-val for classes of ' + item + ' is', p_val, ' and ', round(effect_size, 5))
        if p_val <= 0.01:
            keep.append(item)
    print(keep)
    return keep 

In [27]:
#cat_var_why = my_anova(cat_var)

In [24]:
cat_var_why

NameError: name 'cat_var_why' is not defined

In [20]:
print('Starting with ' + str(len(cat_var)) + ' categorical variables.')

print('Reducing to ' + str(len(cat_var_trim)) + ' categorical variables.')

Starting with 38 categorical variables.


NameError: name 'cat_var_trim' is not defined

<p><a name="PRE"></a></p>

## Preprocessing Data for Models

In [28]:
cat_var = train.select_dtypes('O').columns.tolist()
num_var = train.select_dtypes('number').columns.tolist()
print(len(num_var),' numerical variables')
print(len(cat_var),' categorical variables')

30  numerical variables
38  categorical variables


In [29]:
SalePrice = train['SalePrice']
log_Price = train['log_SalePrice']

In [30]:
train.drop('SalePrice', axis = 1, inplace = True)
train.drop('log_SalePrice', axis = 1, inplace = True)

### Linear Models
#### Dummification of Categorical Variables

In [31]:
preproc_raw = pd.get_dummies(train, columns = cat_var, drop_first=True)
#preproc_trim = pd.get_dummies(train, columns = cat_var_trim, drop_first=True)

In [34]:
print(train.shape)
print(preproc_raw.shape)
#print(preproc_trim.shape)

(1451, 66)
(1451, 188)


In [35]:
preproc_raw.head()

,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MoSold,AgeHome,YrsSnRmdl,TotBthrm,TotFloors,HasAlley,HasPool,NearPos,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NoBasement,BsmtQual_TA,BsmtCond_Gd,BsmtCond_NoBasement,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_NoBasement,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NoBasement,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NoBasement,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_other,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_other,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NoFireplace,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NoGarage,GarageFinish_NoGarage,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_NoGarage,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_NoGarage,GarageCond_Po,GarageCond_TA,PavedDrive_other,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,NeighbQual_Fa,NeighbQual_Gd,NeighbQual_Po,NeighbQual_TA,HasFence_1,HasGarage_1,NearNeg_1
0,65,8450,7,5,196,706,0,150,856,854,0,1710,3,1,2,0,61,0,0,0,2,5,5,3.5,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0
1,80,9600,6,8,0,978,0,284,1262,0,0,1262,3,1,2,298,0,0,0,0,5,31,31,2.5,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1
2,68,11250,7,5,162,486,0,434,920,866,0,1786,3,1,2,0,42,0,0,0,9,7,6,3.5,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0

In [38]:
from sklearn.model_selection import train_test_split

X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
preproc_raw, log_Price, test_size = 0.3, random_state = 22)

In [39]:
X_train_raw.head()

,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MoSold,AgeHome,YrsSnRmdl,TotBthrm,TotFloors,HasAlley,HasPool,NearPos,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NoBasement,BsmtQual_TA,BsmtCond_Gd,BsmtCond_NoBasement,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_NoBasement,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NoBasement,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NoBasement,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_other,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_other,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NoFireplace,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NoGarage,GarageFinish_NoGarage,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_NoGarage,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_NoGarage,GarageCond_Po,GarageCond_TA,PavedDrive_other,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,NeighbQual_Fa,NeighbQual_Gd,NeighbQual_Po,NeighbQual_TA,HasFence_1,HasGarage_1,NearNeg_1
1093,82,11880,7,5,206,704,0,567,1271,0,0,1601,3,1,2,0,0,0,0,0,4,31,31,2.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1
798,75,9000,5,5,0,812,0,124,936,0,0,1128,2,1,1,0,0,0,0,0,6,52,52,1.0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0
1450,75,9937,5,6,0,830,290,136,1256,0,0,1256,3,1,1,736,68,0,0,0,6,43,43,2.5,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0

##### Standardizing numerical variables

In [41]:
nlist = list(train.select_dtypes('number'))
print('Numerical Column Names: \n', num_var)

num_indx = []
for num in num_var:
    num_indx.append(train.select_dtypes('number').columns.get_loc(nvar))
    
print('\nNumerical Columns Indices: \n', num_indx)

Numerical Column Names: 
 ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BedroomAbvGr', 'KitchenAbvGr', 'GarageCars', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'MoSold', 'SalePrice', 'log_SalePrice', 'AgeHome', 'YrsSnRmdl', 'TotBthrm', 'TotFloors', 'HasAlley', 'HasPool', 'NearPos']

Numerical Columns Indices: 
 [27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27]


In [ ]:
num_indx

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

t = [('num', StandardScaler(), num_indx)]
transf = ColumnTransformer(transformers = t,
                          remainder = 'passthrough')

In [ ]:
num_indx

In [ ]:
X_train_scaled = transf.fit_transform(X_train)
X_test_scaled = transf.transform(X_test)

In [ ]:
X_train_scaled.shape

In [ ]:
def look_num(varlist):
    count = 0
    for var in varlist:
        print(var)
        x = X_train_scaled[:,count]
        y = y_train
        print('*'*50)
            
        #info about pearson's correlation
        corr = pearsonr(x, y)
        print('Pearsons correlation (r):', round(corr[0],3))
        count += 1
        print('*'*50)
        
        #Figures
        sns.set_theme(style="whitegrid")
        
        #seaborn Histogram
        f, ax = plt.subplots(1, 2, figsize = (10,6))
        sns.histplot(x=x, data=X_train_scaled, ax = ax[0])
        plt.xticks(rotation = 45)
        
        #seaborn Scatterplot
        sns.scatterplot(x=x, y = y, ax = ax[1])
        plt.xticks(rotation=45)
        
        yield f.show()

In [ ]:
view_num = look_num(nlist)

In [ ]:
next(view_num)

# TESTING


In [ ]:
def get_residuals(model, X, Y):
    
    residuals = Y - model.predict(X)
    
    return residuals

In [ ]:
def model_evaluate(model, X, Y):
    import numpy as np
    from sklearn.metrics import mean_squared_error, r2_score

    rmse = mean_squared_error(Y, 
                            model.predict(X), 
                            squared = False)
    r2 = r2_score(Y, model.predict(X))
    
    if len(np.where(model.predict(X)>20)[0]) > 0: # specific to this dataset
        # Issue, model prediction is too large
        rmse_dollars = np.nan
    else:
        rmse_dollars = mean_squared_error(np.exp(Y), 
                            np.exp(model.predict(X)), 
                            squared = False)

    print('RMSE Log Sale Price: ', rmse)
    print('R^2: ', r2)
    print('RMSE: $', rmse_dollars)
    
    return rmse, r2, rmse_dollars

In [ ]:
def plot_the_model(model, x_train, y_train, x_test, y_test):
    import numpy as np
    from matplotlib import pyplot as plt

    fig, axs = plt.subplots(1,2) 
    fig.set_figheight(5)
    fig.set_figwidth(10)

    x = np.linspace(10, 14.5, 100)
    y = np.linspace(10, 14.5, 100)
    
    # --------------------------------
    # Plot the Actual vs. Predicted
    # --------------------------------
    axs[0].scatter(model.predict(x_train), 
         y_train, color='k')
    axs[0].plot(x,y, 'r-')
 
    axs[0].set_ylabel('Actual log of Sale Price')
    axs[0].set_xlabel('Predicted log of Sale Price')
    axs[0].tick_params(axis='x', rotation=45)
    axs[0].legend(['line','Train'])
    
    axs[1].scatter(model.predict(x_test), 
         y_test, color='teal') 
    axs[1].plot(x,y, 'r-')

    axs[1].set_ylabel('Actual log of Sale Price')
    axs[1].set_xlabel('Predicted log of Sale Price')
    axs[1].tick_params(axis='x', rotation=45)
    axs[1].legend(['line','Test'])

In [ ]:
def get_linear_regress_model(X, Y):
    from sklearn.linear_model import LinearRegression
    lm = LinearRegression()
    
    model = lm.fit(X, Y)
    print('LM model object created.')
    
    return model

In [ ]:
def get_lasso_model(X, Y):
    from sklearn.linear_model import Lasso
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import KFold

    skfold = KFold(n_splits=5, shuffle=True, random_state=0)
    lasso_gs  = Lasso()
    lasso_gs.set_params(normalize=False)

    params = {
        "alpha": [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]
    }

    gs = GridSearchCV(lasso_gs, params, scoring = 'r2', cv=skfold)
    gs.fit(X, Y)
    
    model = gs.best_estimator_
    
    print('Lasso model object created.')
    print(gs.best_estimator_)
    
    return model

In [ ]:
def get_ridge_model(X, Y):
    from sklearn.linear_model import Ridge
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import KFold

    skfold = KFold(n_splits=3, shuffle=True, random_state=0)
    ridge_gs  = Ridge()
    ridge_gs.set_params(normalize=False)

    params = {
        "alpha": [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]
    }

    gs = GridSearchCV(ridge_gs, params, scoring = 'r2', cv=skfold)
    gs.fit(X, Y)
    
    model = gs.best_estimator_
    
    print('Ridge model object created.')
    print(gs.best_estimator_)
    
    return model

In [ ]:
def get_elasticNet_model(X, Y):
    from sklearn.linear_model import ElasticNet
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import KFold

    skfold = KFold(n_splits=3, shuffle=True, random_state=0)
    elastic_gs  = ElasticNet()
    elastic_gs.set_params(normalize=False)

    params = [{
        "alpha": [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01],
        "l1_ratio":[0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90]
    }]

    gs = GridSearchCV(elastic_gs, params, scoring = 'r2', cv=skfold)
    gs.fit(X_train_scaled, y_train)
    
    model = gs.best_estimator_
    
    print('Elastic Net model object created.')
    print(gs.best_estimator_)
    
    return model

In [ ]:
# initializing these variables for model comparision
model_descrp = []
r2_log_train = []
r2_log_test = []
rmse_log_train = []
rmse_log_test = []
rmse_dollars_train = []
rmse_dollars_test = []

In [ ]:
def append_myMetrics(which_model, r2_train, r2_test, rmse_train, rmse_test, rmse_train_dollars, rmse_test_dollars):
    
    r2_log_train_temp = r2_train
    r2_log_test_temp = r2_test
    rmse_log_train_temp = rmse_train
    rmse_log_test_temp = rmse_test
    rmse_dollars_train_temp = rmse_train_dollars
    rmse_dollars_test_temp = rmse_test_dollars

    model_descrp.append(which_model)
    if r2_log_train_temp <= 1 and r2_log_train_temp > 0:
        r2_log_train.append(r2_log_train_temp)
    elif r2_log_train_temp > 1:   
        r2_log_train.append(1000)
    else:
        r2_log_train.append(-1000)

    if r2_log_test_temp <= 1 and r2_log_test_temp > 0:
        r2_log_test.append(r2_log_test_temp)
    elif r2_log_test_temp > 1:   
        r2_log_test.append(1000)
    else:
        r2_log_test.append(-1000)
    
    if rmse_log_train_temp <= 10:
        rmse_log_train.append(rmse_log_train_temp)
    elif rmse_log_train_temp > 10:   
        rmse_log_train.append(1000)

    if rmse_log_test_temp <= 10:
        rmse_log_test.append(rmse_log_test_temp)
    elif rmse_log_test_temp > 10:   
        rmse_log_test.append(1000)
    
    rmse_dollars_train.append(rmse_dollars_train_temp)
    rmse_dollars_test.append(rmse_dollars_test_temp)

In [ ]:
#linear regression model
lm_model = get_linear_regress_model(X_train_scaled, y_train)

In [ ]:
rmse_train, r2_train, rmse_train_dollars= model_evaluate(lm_model, X_train_scaled, y_train)

In [ ]:
rmse_test, r2_test, rmse_test_dollars= model_evaluate(lm_model, X_test_scaled, y_test)

In [ ]:
residuals_train = get_residuals(lm_model, X_train_scaled, y_train)
plt.hist(residuals_train, 100, facecolor = 'teal');
plt.title('Histograme of Residuals - Training Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
residuals_test = get_residuals(lm_model, X_test_scaled, y_test)
plt.hist(residuals_test, 100, facecolor = 'magenta');
plt.title('Histograme of Residuals - Test Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
print('Linear Regression Model - No Regularization');
plot_the_model(lm_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
which_model = 'Linear Regression'
append_myMetrics(which_model, r2_train, r2_test, rmse_train, rmse_test, rmse_train_dollars, rmse_test_dollars)

In [ ]:
#Lasso

In [ ]:
from sklearn.linear_model import Lasso
lasso  = Lasso()
lasso.set_params(normalize=False)
lasso_scores_train = []
lasso_scores_test  = []

alphas = np.linspace(0.0001,0.05,100)
coefs_lasso  = []

for alpha in alphas:
        lasso.set_params(alpha=alpha)
        lasso.fit(X_train_scaled, y_train)  
        coefs_lasso.append(lasso.coef_)
        
        lasso_scores_train.append(lasso.score(X_train_scaled, y_train))
        lasso_scores_test.append(lasso.score(X_test_scaled, y_test))

coefs_lasso = pd.DataFrame(coefs_lasso, index = alphas, columns = preproc.columns)  
# coefs_lasso.head()

In [ ]:
lasso_scores_train = np.array(lasso_scores_train) 
lasso_scores_test  = np.array(lasso_scores_test)

In [ ]:
plt.plot(alphas, lasso_scores_train, label=r'$train\ R^2$')
plt.plot(alphas, lasso_scores_test, label=r'$test\ R^2$')
plt.title(r'Lasso Train-Test $R^2$ Comparison')

lasso_underfit = lasso_scores_train < lasso_scores_test
first_underfit  = np.min(alphas[lasso_underfit])
print('lambda = ', first_underfit)
plt.axvline(first_underfit, linestyle='--', color='g', label='optimal lambda', alpha=0.4)

plt.legend(loc=1)
plt.xlabel(r'hyperparameter $\lambda$')
plt.ylabel(r'$R^2$')

In [ ]:
#Lasso 2.0

In [ ]:
lasso_model = get_lasso_model(X_train_scaled, y_train)

In [ ]:
rmse_train, r2_train, rmse_train_dollars= model_evaluate(lasso_model, X_train_scaled, y_train)

In [ ]:
rmse_test, r2_test, rmse_test_dollars= model_evaluate(lasso_model, X_test_scaled, y_test)

In [ ]:
residuals_train = get_residuals(lasso_model, X_train_scaled, y_train)
plt.hist(residuals_train, 100, facecolor = 'black');
plt.title('Histograme of Residuals - Training Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
residuals_test = get_residuals(lasso_model, X_test_scaled, y_test)
plt.hist(residuals_test, 100, facecolor = 'magenta');
plt.title('Histograme of Residuals - Test Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
print('Linear Regression Model - Lasso');
plot_the_model(lasso_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
which_model = 'Lasso'
append_myMetrics(which_model, r2_train, r2_test, rmse_train, rmse_test, rmse_train_dollars, rmse_test_dollars)

In [ ]:
#Ridge

In [ ]:
ridge_model = get_ridge_model(X_train_scaled, y_train)

In [ ]:
rmse_train, r2_train, rmse_train_dollars= model_evaluate(ridge_model, X_train_scaled, y_train)

In [ ]:
rmse_test, r2_test, rmse_test_dollars= model_evaluate(ridge_model, X_test_scaled, y_test)

In [ ]:
residuals_train = get_residuals(ridge_model, X_train_scaled, y_train)
plt.hist(residuals_train, 100, facecolor = 'black');
plt.title('Histograme of Residuals - Training Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
residuals_test = get_residuals(ridge_model, X_test_scaled, y_test)
plt.hist(residuals_test, 100, facecolor = 'magenta');
plt.title('Histograme of Residuals - Test Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
print('Linear Regression Model - Ridge');
plot_the_model(ridge_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
which_model = 'Ridge'
append_myMetrics(which_model, r2_train, r2_test, rmse_train, rmse_test, rmse_train_dollars, rmse_test_dollars)

In [ ]:
#Elastic Net

In [ ]:
EN_model = get_elasticNet_model(X_train_scaled, y_train)

In [ ]:
rmse_train, r2_train, rmse_train_dollars= model_evaluate(EN_model, X_train_scaled, y_train)

In [ ]:
rmse_test, r2_test, rmse_test_dollars= model_evaluate(EN_model, X_test_scaled, y_test)

In [ ]:
residuals_train = get_residuals(EN_model, X_train_scaled, y_train)
plt.hist(residuals_train, 100, facecolor = 'black');
plt.title('Histograme of Residuals - Training Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
residuals_test = get_residuals(EN_model, X_test_scaled, y_test)
plt.hist(residuals_test, 100, facecolor = 'magenta');
plt.title('Histograme of Residuals - Test Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
print('Linear Regression Model - Elastic Net');
plot_the_model(EN_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
which_model = 'Elastic Net'
append_myMetrics(which_model, r2_train, r2_test, rmse_train, rmse_test, rmse_train_dollars, rmse_test_dollars)

In [ ]:
pd.options.display.float_format = '{:,.4f}'.format
zipped = list(zip(model_descrp,
                 rmse_log_train, rmse_log_test, 
                 rmse_dollars_train, rmse_dollars_test,
                 r2_log_train, r2_log_test))

results = pd.DataFrame(zipped, columns=['model',
                                       'rmse log train',
                                       'rmse log test',
                                       'rmse train',
                                       'rmse test',
                                       'r2 train',
                                       'r2 test'])

#results.head(10)
results_sorted = results.sort_values('rmse log test', ascending = True)
results_sorted.head(10)

In [ ]:
EN_model.get_params

In [ ]:
EN_model.intercept_

In [ ]:
coefs = pd.DataFrame(
    EN_model.coef_,
    columns=["Coefficients"],
    index=preproc.columns,
)
coefs.head()

In [ ]:
coefs.head(25)

### Non Linear Models
#### Label Encoding of Categorical Variables

### All Models
#### Standardization of numerical variables
#### Split train and validate datasets

<p><a name="EXP"></a></p>

## Exporting Cleaned Datasets

Based on our first pass, and familiarization with the data, the independent variables in this dataset fall broadly into these 'buckets':
- Land and Exterior - neighborhood, lot and land characteristics
- House Quality, Construction and Utilities- year built, types of materials used and quality
- Indoor Areas
- Basement
- Semi-Indoor Areas - Garage, Porches, Pool, Fence
- Sale Information - month, year, type of sale etc

We will look at the categorical variables in each of these buckets individually and adjust features.

In [ ]:
#initializing an iterator to help bucket variables
call_var = iter(new_var_list)

In [ ]:
next(call_var)

In [ ]:
#Bucketing variables into lists
Location = ['LotFrontage', 'LotArea', 'NearNeg', 'MSZoning','LotShape', 'LandContour', 'LotConfig', 'NeighbQual']
HouseQual = ['OverallQual', 'MasVnrArea', 'AgeHome', 'YrsSnRmdl', 'HasFireplace', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation']
Basement = ['BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu']
IndoorA = ['1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BedroomAbvGr', 'TotRmsAbvGrd', 'TotBthrm', ]
GaragePlus = ['WoodDeckSF', 'OpenPorchSF','HasFence', 'HasGarage', 'GarageType', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive']
SaleInfo = ['SaleType', 'SaleCondition']

In [42]:
#Sanity Check to make sure I got all the variables
bucketlist = [Location, HouseQual, Basement, IndoorA, GaragePlus, SaleInfo]
temp_ = 0
for L in bucketlist:
    temp_ += len(L)
print(temp_)
print(len(new_var_list)) #expect this to be one longer, for dependent variable

NameError: name 'Location' is not defined

In [ ]:
#Removing MSZoning = C observations #9 obs
obs_to_drop = np.where(train['MSZoning'] == 'C (all)')[0].tolist()